In [1]:
library("geneplotter")
library("ggplot2")
library("vsn")
library("genefilter")
library("DESeq2")
getwd()

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain intro

[1] "/home/gstupp/projects/Wolan/ana_extractions/analysis/H1"

In [2]:
meta_path = "metadata.csv"
sampleTable = read.csv(meta_path, row.names = 1)
sampleTable$technical=factor(sampleTable$technical)
sampleTable$sample_type=factor(sampleTable$sample_type)
sampleTable$sample_type <- relevel(sampleTable$sample_type, ref="whole")
sampleTable

,technical,sample_type,sample_name,path
H1_PelletTriflic_Fl_R1,1,sonic_pellet,H1_PelletTriflic_Fl,data/H1_PelletTriflic_FlFusion_5ppm_10ppmfrag/H1_PelletTriflic_Fl_R1.txt
H1_PelletTriflic_Fl_R2,2,sonic_pellet,H1_PelletTriflic_Fl,data/H1_PelletTriflic_FlFusion_5ppm_10ppmfrag/H1_PelletTriflic_Fl_R2.txt
H1_PelletTriflic_Fl_R3,3,sonic_pellet,H1_PelletTriflic_Fl,data/H1_PelletTriflic_FlFusion_5ppm_10ppmfrag/H1_PelletTriflic_Fl_R3.txt
H1_SonicSupTriflic_V2_R1,1,sonic_sup,H1_SonicSupTriflic_V2,data/H1_SonicSupTriflic_FlFusion_5ppm_10ppmfrag/H1_SonicSupTriflic_V2_R1.txt
H1_SonicSupTriflic_V2_R2,2,sonic_sup,H1_SonicSupTriflic_V2,data/H1_SonicSupTriflic_FlFusion_5ppm_10ppmfrag/H1_SonicSupTriflic_V2_R2.txt
H1_SonicSupTriflic_V2_R3,3,sonic_sup,H1_SonicSupTriflic_V2,data/H1_SonicSupTriflic_FlFusion_5ppm_10ppmfrag/H1_SonicSupTriflic_V2_R3.txt
H1_wholetriflic_R1,1,whole,H1_wholetriflic,data/H1_WholeTriflic_FlFusion_5ppm_10ppmfrag/H1_wholetriflic_R1.txt
H1_wholetriflic_R2,2,whole,H1_wholetriflic,data/H1_WholeTriflic_FlFusion_5ppm_10ppmfrag/H1_wholetriflic_R2.txt
H1_wholetriflic_R3,3,whole,H1_wholetriflic,data/H1_WholeTriflic_FlFusion_5ppm_10ppmfrag/H1_wholetriflic_R3.txt


In [3]:
countData = read.csv("out/df_filt.csv", row.names = 1)
colnames(countData) = lapply(colnames(countData), function(x) {gsub("\\.", "_", x)})
locusTable = read.csv("out/locus_df.csv", row.names = 1)
sampleTable = sampleTable[colnames(countData),]

In [4]:
head(locusTable)

all_ids                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
202956 448405;857983                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [5]:
dds = DESeqDataSetFromMatrix(countData, colData = sampleTable, ~ sample_type)

## collapse replicates
#dds = collapseReplicates(dds, dds$sample_name)
#sampleTable = colData(dds)

In [6]:
head(counts(dds))

,H1_PelletTriflic_Fl_R1,H1_PelletTriflic_Fl_R2,H1_PelletTriflic_Fl_R3,H1_SonicSupTriflic_V2_R1,H1_SonicSupTriflic_V2_R2,H1_SonicSupTriflic_V2_R3,H1_wholetriflic_R1,H1_wholetriflic_R2,H1_wholetriflic_R3
202956,9,7,6,9,10,11,13,10,11
209005,32,33,34,15,15,19,25,31,34
217964,2,2,3,0,0,0,6,5,6
248190,3,1,3,0,0,0,3,5,5
251275,8,3,2,0,0,0,3,3,6
265827,5,8,10,0,0,0,1,0,0


In [7]:
dds = DESeq(dds)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.
final dispersion estimates
fitting model and testing


In [8]:
#mcols(dds)<-cbind(mcols(dds),locusTable)
save(dds, file = "out/dds")

In [9]:
res = results(dds, contrast=c("sample_type","sonic_pellet","whole"))
summary(res)


out of 8083 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 600, 7.4% 
LFC < 0 (down)   : 458, 5.7% 
outliers [1]     : 0, 0% 
low counts [2]   : 0, 0% 
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [10]:
res = cbind(res, locusTable)
resOrdered <- res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="out/sonic_pellet_v_whole_deseq_results.csv")

In [11]:
res = results(dds, contrast=c("sample_type","whole","sonic_sup"))
summary(res)


out of 8083 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 3953, 49% 
LFC < 0 (down)   : 1851, 23% 
outliers [1]     : 0, 0% 
low counts [2]   : 313, 3.9% 
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [12]:
res = cbind(res, locusTable)
resOrdered <- res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="out/sonic_whole_v_sup_deseq_results.csv")

In [13]:
res = results(dds, contrast=c("sample_type","sonic_pellet","sonic_sup"))
summary(res)


out of 8083 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 3941, 49% 
LFC < 0 (down)   : 1720, 21% 
outliers [1]     : 0, 0% 
low counts [2]   : 313, 3.9% 
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [14]:
res = cbind(res, locusTable)
resOrdered <- res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="out/sonic_pellet_v_sup_deseq_results.csv")